In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
from tqdm.auto import tqdm
import datetime

tqdm.pandas()

def get_item_info(item_code):
    item_url = f'https://finance.naver.com/item/main.nhn?code={item_code}'
    tables = pd.read_html(item_url, encoding='cp949')
    item_info = []
    finance_info = tables[3].iloc[:, [0, -1]]
    finance_info.columns = [0, 1]
    finance_info
    for t in tables:
    # [1]은 column을 의미
        if t.shape[1] == 2:
            item_info.append(t)
    df_item = pd.concat(item_info)
    df_item = df_item.set_index(0).T
    time.sleep(0.1)
    return df_item

In [5]:
# 업종: 제약
url = 'https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=261'

table = pd.read_html(url, encoding='cp949')

raw = table[2]
# 전체가 NaN일 때만,      컬럼
raw = raw.dropna(how='all').dropna(axis=1, how='all')
raw['종목명_전처리'] = raw['종목명'].str.replace('*', '')
raw['종목명_전처리'] = raw['종목명_전처리'].str.strip()

import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')
df_krx.head()

df_item_code_name = df_krx[['Symbol', 'Name']]
df_item_code_name.columns = ['종목코드', '종목명_전처리']
df_item_code_name.head()

df = raw.merge(df_item_code_name, on='종목명_전처리', how='left')
df.head()

C:\Users\ysn65\AppData\Local\Temp/ipykernel_18600/3079302243.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  raw['종목명_전처리'] = raw['종목명'].str.replace('*', '')


,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,종목명_전처리,종목코드
0,한국파마 *,78100.0,8600.0,+12.37%,78100.0,78200.0,3588899.0,279786.0,688406.0,한국파마,032300
1,삼아제약 *,23050.0,1550.0,+7.21%,23000.0,23050.0,226300.0,5151.0,219443.0,삼아제약,009300
2,코스맥스엔비티 *,9210.0,580.0,+6.72%,9200.0,9210.0,191697.0,1733.0,127164.0,코스맥스엔비티,222040
3,지엘팜텍 *,1455.0,85.0,+6.20%,1450.0,1455.0,6712248.0,9666.0,12967049.0,지엘팜텍,204840
4,이수앱지스 *,16350.0,950.0,+6.17%,16350.0,16400.0,1619361.0,25924.0,492397.0,이수앱지스,086890


In [10]:
result = df['종목코드'].progress_apply(get_item_info)

  0%|          | 0/149 [00:00<?, ?it/s]

In [11]:
result

0      0       시가총액   시가총액순위     상장주식수     액면가l매매단위 외...
1      0       시가총액    시가총액순위    상장주식수       액면가l매매단위...
2      0       시가총액    시가총액순위     상장주식수     액면가l매매단위 ...
3      0     시가총액     시가총액순위     상장주식수     액면가l매매단위 외...
4      0       시가총액    시가총액순위     상장주식수     액면가l매매단위 ...
                             ...                        
144    0       시가총액    시가총액순위     상장주식수     액면가l매매단위 ...
145    0     시가총액    시가총액순위   상장주식수     액면가l매매단위 외국인한...
146    0           시가총액   시가총액순위     상장주식수       액면가l...
147    0         시가총액    시가총액순위     상장주식수 외국인한도주식수(A)...
148    0       시가총액    시가총액순위    상장주식수     액면가l매매단위 외...
Name: 종목코드, Length: 149, dtype: object

In [12]:
result.tolist()

[0       시가총액   시가총액순위     상장주식수     액면가l매매단위 외국인한도주식수(A) 외국인보유주식수(B)  \
 1  8,518  억원  코스닥 87위  10906701  500원  l  1주    10906701       37722   
 
 0 외국인소진율(B/A)    투자의견l목표주가           52주최고l최저  PERlEPS(2021.06)    추정PERlEPS  \
 1       0.35%  N/A  l  N/A  94,800  l  20,000  169.41배  l  461원  N/A  l  N/A   
 
 0  PBRlBPS (2021.06) 배당수익률l2020.12 동일업종 PER 동일업종 등락률  
 1  17.16배  l  4,551원         0.06%  107.03배   +0.01%  ,
 0       시가총액    시가총액순위    상장주식수       액면가l매매단위 외국인한도주식수(A) 외국인보유주식수(B)  \
 1  1,468  억원  코스닥 707위  6370000  1,000원  l  1주     6370000       77432   
 
 0 외국인소진율(B/A)    투자의견l목표주가           52주최고l최저 PERlEPS(2021.06)    추정PERlEPS  \
 1       1.22%  N/A  l  N/A  23,000  l  14,300    N/A  l  -175원  N/A  l  N/A   
 
 0  PBRlBPS (2021.06) 배당수익률l2020.12 동일업종 PER 동일업종 등락률  
 1  0.84배  l  27,352원         1.30%  107.03배   +0.01%  ,
 0       시가총액    시가총액순위     상장주식수     액면가l매매단위 외국인한도주식수(A) 외국인보유주식수(B)  \
 1  1,900  억원  코스닥 569위  20628000  500원  l  1주    20628000      300027   


In [14]:
df_item_info = pd.concat(result.tolist())
df_item_info

,시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,PERlEPS(2021.06),...,배당수익률,PERlEPS(2020.12),PBRlBPS (2020.12),PERlEPS(2021.03),PBRlBPS (2021.03),PERlEPS(2021.05),PBRlBPS (2021.05),배당수익률l2020.11,PERlEPS(2020.06),PBRlBPS (2020.06)
1,"8,518 억원",코스닥 87위,10906701,500원 l 1주,10906701,37722,0.35%,N/A l N/A,"94,800 l 20,000",169.41배 l 461원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,468 억원",코스닥 707위,6370000,"1,000원 l 1주",6370000,77432,1.22%,N/A l N/A,"23,000 l 14,300",N/A l -175원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,900 억원",코스닥 569위,20628000,500원 l 1주,20628000,300027,1.45%,"4.00매수 l 14,950","12,900 l 7,000",N/A l -449원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,755 억원,코스닥 1169위,51886844,100원 l 1주,51886844,577674,1.11%,N/A l N/A,"2,390 l 855",N/A l -67원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"5,425 억원",코스닥 164위,33182668,500원 l 1주,33182668,102147,0.31%,N/A l N/A,"25,000 l 6,660",N/A l -548원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,"7,152 억원",코스닥 103위,47206389,500원 l 1주,47206389,10209006,21.63%,N/A l N/A,"21,550 l 11,700","N/A l -1,133원",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,623 억원,코스피 986위,845990,500원 l 1주,845990,591,0.07%,N/A l N/A,"237,000 l 61,600","22.73배 l 3,242원",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"4조 1,370 억원",코스피 82위,11686538,"5,000원 l 1주",11686538,2519699,21.56%,"4.00매수 l 457,143","538,000 l 209,000","44.26배 l 7,998원",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2조 703 억원,코스피 152위,60096155,NaN,60096155,38571511,64.18%,"4.00매수 l 40,000","54,000 l 24,200",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A l -336원,"3.61배 l 9,552원"


In [16]:
df_item_info = df_item_info.reset_index()
df_item_info['종목코드'] = df['종목코드']
del df_item_info['index']
df_item_info

,level_0,시가총액,시가총액순위,상장주식수,액면가l매매단위,외국인한도주식수(A),외국인보유주식수(B),외국인소진율(B/A),투자의견l목표주가,52주최고l최저,...,PERlEPS(2020.12),PBRlBPS (2020.12),PERlEPS(2021.03),PBRlBPS (2021.03),PERlEPS(2021.05),PBRlBPS (2021.05),배당수익률l2020.11,PERlEPS(2020.06),PBRlBPS (2020.06),종목코드
0,0,"8,518 억원",코스닥 87위,10906701,500원 l 1주,10906701,37722,0.35%,N/A l N/A,"94,800 l 20,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,032300
1,1,"1,468 억원",코스닥 707위,6370000,"1,000원 l 1주",6370000,77432,1.22%,N/A l N/A,"23,000 l 14,300",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,009300
2,2,"1,900 억원",코스닥 569위,20628000,500원 l 1주,20628000,300027,1.45%,"4.00매수 l 14,950","12,900 l 7,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222040
3,3,755 억원,코스닥 1169위,51886844,100원 l 1주,51886844,577674,1.11%,N/A l N/A,"2,390 l 855",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,204840
4,4,"5,425 억원",코스닥 164위,33182668,500원 l 1주,33182668,102147,0.31%,N/A l N/A,"25,000 l 6,660",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,086890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,144,"7,152 억원",코스닥 103위,47206389,500원 l 1주,47206389,10209006,21.63%,N/A l N/A,"21,550 l 11,700",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,334970
145,145,623 억원,코스피 986위,845990,500원 l 1주,845990,591,0.07%,N/A l N/A,"237,000 l 61,600",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,005257
146,146,"4조 1,370 억원",코스피 82위,11686538,"5,000원 l 1주",11686538,2519699,21.56%,"4.00매수 l 457,143","538,000 l 209,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,006280
147,147,2조 703 억원,코스피 152위,60096155,NaN,60096155,38571511,64.18%,"4.00매수 l 40,000","54,000 l 24,200",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A l -336원,"3.61배 l 9,552원",950210


In [19]:
df_info = df.merge(df_item_info)
df_info.head()

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,종목명_전처리,...,배당수익률,PERlEPS(2020.12),PBRlBPS (2020.12),PERlEPS(2021.03),PBRlBPS (2021.03),PERlEPS(2021.05),PBRlBPS (2021.05),배당수익률l2020.11,PERlEPS(2020.06),PBRlBPS (2020.06)
0,한국파마 *,78100.0,8600.0,+12.37%,78100.0,78200.0,3588899.0,279786.0,688406.0,한국파마,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,삼아제약 *,23050.0,1550.0,+7.21%,23000.0,23050.0,226300.0,5151.0,219443.0,삼아제약,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,코스맥스엔비티 *,9210.0,580.0,+6.72%,9200.0,9210.0,191697.0,1733.0,127164.0,코스맥스엔비티,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,지엘팜텍 *,1455.0,85.0,+6.20%,1450.0,1455.0,6712248.0,9666.0,12967049.0,지엘팜텍,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,이수앱지스 *,16350.0,950.0,+6.17%,16350.0,16400.0,1619361.0,25924.0,492397.0,이수앱지스,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.shape, df_item_info.shape, df_info.shape

((149, 11), (149, 27), (149, 37))

In [25]:
today = datetime.datetime.today()
today = today.strftime('%Y-%m-%d')
today

file_name_1 = url.split('=')[1].replace('&', '_')
file_name_2 = url.split('=')[2]
file_name = f'{file_name_1}_{file_name_2}_{today}.csv'

file_name

'upjong_no_261_2021-09-11.csv'

In [27]:
df_info.to_csv(file_name, index=False)
pd.read_csv(file_name)

,종목명,현재가,전일비,등락률,매수호가,매도호가,거래량,거래대금,전일거래량,종목명_전처리,...,배당수익률,PERlEPS(2020.12),PBRlBPS (2020.12),PERlEPS(2021.03),PBRlBPS (2021.03),PERlEPS(2021.05),PBRlBPS (2021.05),배당수익률l2020.11,PERlEPS(2020.06),PBRlBPS (2020.06)
0,한국파마 *,78100.0,8600.0,+12.37%,78100.0,78200.0,3588899.0,279786.0,688406.0,한국파마,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,삼아제약 *,23050.0,1550.0,+7.21%,23000.0,23050.0,226300.0,5151.0,219443.0,삼아제약,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,코스맥스엔비티 *,9210.0,580.0,+6.72%,9200.0,9210.0,191697.0,1733.0,127164.0,코스맥스엔비티,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,지엘팜텍 *,1455.0,85.0,+6.20%,1450.0,1455.0,6712248.0,9666.0,12967049.0,지엘팜텍,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,이수앱지스 *,16350.0,950.0,+6.17%,16350.0,16400.0,1619361.0,25924.0,492397.0,이수앱지스,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,프레스티지바이오로직스 *,15150.0,1050.0,-6.48%,15150.0,15200.0,625678.0,9678.0,501201.0,프레스티지바이오로직스,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,녹십자홀딩스2우,73700.0,5300.0,-6.71%,73700.0,74100.0,4719.0,356.0,2832.0,녹십자홀딩스2우,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,녹십자,354000.0,25500.0,-6.72%,354000.0,354500.0,339546.0,120234.0,133432.0,녹십자,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,프레스티지바이오파마,34450.0,2750.0,-7.39%,34400.0,34450.0,1137115.0,39909.0,669255.0,프레스티지바이오파마,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N/A l -336원,"3.61배 l 9,552원"
